In [4]:
import yfinance as yf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Attention
from sklearn.preprocessing import MinMaxScaler


In [5]:
# Veri çekme
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')
close_prices = data['Close'].values.reshape(-1, 1)

[*********************100%%**********************]  1 of 1 completed


In [6]:
# Normalizasyon
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)


In [7]:
# Seq2Seq veri seti oluşturma
def create_dataset(dataset, n_past, n_future):
    X, y = [], []
    for i in range(n_past, len(dataset) - n_future +1):
        X.append(dataset[i - n_past:i, 0])
        y.append(dataset[i:i + n_future, 0])
    return np.array(X), np.array(y)


In [8]:
n_past = 60
n_future = 1
X, y = create_dataset(close_prices_scaled, n_past, n_future)
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

In [9]:
# Encoder
encoder_inputs = Input(shape=(n_past, 1))
encoder_lstm = LSTM(100, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Dikkat Mekanizması
attention = Attention(use_scale=True)
attention_output = attention([encoder_outputs, encoder_outputs])

# Decoder
decoder_inputs = Input(shape=(n_future, 1))
decoder_lstm = LSTM(100, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)


In [10]:
# Dikkat ve Decoder çıktılarını birleştirme
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Dense katmanı
decoder_dense = Dense(1, activation='linear')
decoder_outputs = decoder_dense(decoder_concat_input)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 1, 100), (None, 60, 100)]

In [ ]:

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
decoder_input_data = np.zeros_like(X)
decoder_input_data[:,1:,0] = y[:,:-1,0]
decoder_target_data = y

model.fit([X, decoder_input_data], decoder_target_data, batch_size=32, epochs=10, validation_split=0.2)
